In [1]:
from ansatz_simulation_class import AnsatzSimulation
from genetic_quantum import QuanvLayer, QuantumModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from torch import tensor
import matplotlib.pyplot as plt

### Instantiating genetic ansatz model

In [2]:
n_qubits = 4
patch_size = 2
num_classes = 2
input_size = 28
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['rz_gate', 'pauli_y', 'rx_gate', 'phase']]
quanv = QuanvLayer(n_qubits, patch_size, toy_chromosome, mode='2d')
toy_qhcnn = QuantumModel(n_qubits, num_classes, patch_size, input_size, toy_chromosome)
toy_qhcnn

QuantumModel(
  (quanv_layer): QuanvLayer()
  (fc1): Linear(in_features=2916, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

### Getting data and sampling it

In [3]:
transform = transforms.ToTensor()
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


In [4]:
selected_classes = [0, 1]
reduced_mnist = [mnist_dataset.__getitem__(index) for index in range(mnist_dataset.__len__()) if mnist_dataset.__getitem__(index)[1] in selected_classes]
len(reduced_mnist)

12665

In [5]:
class_one = 0
class_two = 0
binary_dataset = []

for image in reduced_mnist:
    if image[1] == 0 and class_one < 50:
        binary_dataset.append(image)
        class_one +=1 
    elif image[1] == 1 and class_two < 50:
        binary_dataset.append(image)
        class_two += 1
        
    if class_one == 50 and class_two == 50:
        break

In [6]:
mnist_images, mnist_labels = zip(*binary_dataset)

In [7]:
tensor_mnist = torch.stack(mnist_images)
height = tensor_mnist.shape[3]

In [15]:
mnist_labels = tensor(mnist_labels)

In [8]:
tensor_mnist.squeeze(1).shape

torch.Size([100, 28, 28])

In [9]:
output = toy_qhcnn.forward(tensor_mnist.squeeze(1).numpy())

Passing through quanvolution layer...
Patch crafting time: 0.014747400302439928
Image quantum processing time: 22.67309629963711
Quanvolution processing time: 22.90649880003184
Classical layers processing time: 0.0011980002745985985


In [10]:
output

tensor([[-0.5248, -0.8957],
        [-0.5062, -0.9233],
        [-0.5519, -0.8577],
        [-0.5162, -0.9083],
        [-0.5082, -0.9203],
        [-0.5475, -0.8636],
        [-0.5106, -0.9166],
        [-0.5123, -0.9141],
        [-0.5060, -0.9236],
        [-0.4926, -0.9442],
        [-0.4985, -0.9351],
        [-0.5898, -0.8084],
        [-0.6051, -0.7898],
        [-0.5010, -0.9312],
        [-0.5625, -0.8435],
        [-0.5059, -0.9237],
        [-0.5104, -0.9170],
        [-0.5513, -0.8585],
        [-0.5526, -0.8567],
        [-0.5131, -0.9130],
        [-0.5340, -0.8825],
        [-0.5018, -0.9300],
        [-0.5205, -0.9019],
        [-0.5176, -0.9062],
        [-0.5737, -0.8288],
        [-0.5423, -0.8708],
        [-0.5154, -0.9094],
        [-0.5048, -0.9254],
        [-0.5262, -0.8937],
        [-0.5024, -0.9291],
        [-0.4818, -0.9615],
        [-0.5024, -0.9291],
        [-0.4982, -0.9355],
        [-0.5759, -0.8260],
        [-0.5190, -0.9041],
        [-0.5711, -0

### Training my model

In [11]:
from sampled_cv_dataset import SampledDataset4Training

training_zeros_ones = SampledDataset4Training(binary_dataset)

batch_size = 4

training_loader = DataLoader(training_zeros_ones, batch_size, shuffle=True)

In [12]:
training_zeros_ones.__getitem__(0)['image'].shape

torch.Size([1, 28, 28])

In [16]:
optimizer = torch.optim.SGD(toy_qhcnn.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(output, mnist_labels)

In [17]:
loss

tensor(0.7309, grad_fn=<NllLossBackward0>)

In [ ]:
def train_one_epoch(model, epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [14]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

TypeError: tensor or list of tensors expected, got <class 'str'>

### Testing tensor dimensions and quantum convolution

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 4
dummy_input = torch.zeros(input_size, input_size, 2).numpy()
patched_images = [image.extract_patches_2d(img, (patch_size, patch_size)) for img in dummy_input]
#dummy_output = quanv.forward(dummy_input)

tensor(patched_images).shape

torch.Size([4, 3, 2, 2])

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 28
shape = (input_size, input_size)
dummy_input = np.zeros(shape, dtype=float)
patched_images = image.extract_patches_2d(dummy_input, (patch_size, patch_size)) 
#dummy_output = quanv.forward(dummy_input)
n_patches, patch_h, patch_w = tensor(patched_images).shape
patched_images = tensor(patched_images).contiguous().view(n_patches, patch_h*patch_w)
image._extract_patches()

In [ ]:
patched_images[0]

tensor([0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
patched_images.shape

torch.Size([729, 4])

In [ ]:
myAnsatz = AnsatzSimulation(n_qubits)
outputs = [myAnsatz.simulate_circuit(patch, 'rx', toy_chromosome) for patch in patched_images]

In [ ]:
tensor(outputs).shape

torch.Size([1458, 4])